In [ ]:
import os
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from utils import load_rules_from_config, detect_encoding, add_rule  # 这些方法假设存在utils.py中
import gradio as gr

class ModelTrainer:
    def __init__(self, role_json=None, default_file_path="imp/default_login_logs.csv"):
        """
        初始化模型训练类，加载配置文件、设置默认文件路径
        """
        self.role_json = role_json
        self.default_file_path = default_file_path
        self.rules_config = self.load_rules_from_config()
        self.imp_folder = "imp"
        os.makedirs(self.imp_folder, exist_ok=True)
        
    def load_rules_from_config(self):
        """
        从配置文件加载规则
        """
        if self.role_json is None:
            return load_rules_from_config();
        else:
            return self.role_json

    def get_default_file(self, file):
        """
        如果未提供上传文件，则使用默认文件。
        """
        if file is None:
            logging.info("未检测到上传文件，尝试使用默认文件...")
            if not os.path.exists(self.default_file_path):
                raise FileNotFoundError(f"默认文件 {self.default_file_path} 不存在，请上传文件！")
            return self.default_file_path
        return file.name

    def load_and_preprocess_data(self, file):
        """
        读取并预处理数据，返回处理后的 DataFrame
        """
        # 获取文件路径
        file_path = self.get_default_file(file)
        
        # 检测文件编码
        encoding = detect_encoding(file_path)
        logging.info(f"检测到的文件编码为: {encoding}")

        # 根据编码读取文件
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            logging.info("上传文件预览:\n", df.head())  # 打印上传文件的前几行
        except UnicodeDecodeError as e:
            raise ValueError(f"文件读取失败，可能是编码不匹配。检测到的编码为 {encoding}, 请确认文件格式！") from e

        # 数据预处理
        df = self.preprocess_data(df)
        return df, encoding

    def preprocess_data(self, df):
        """
        数据清洗和特征提取
        """
        df["登录时间"] = pd.to_datetime(df["登录时间"], errors="coerce")
        df["时间范围分钟"] = df.groupby("用户ID")["登录时间"].transform(lambda x: max(1, (x.max() - x.min()).total_seconds() / 60))
        df["登录失败次数"] = df.groupby("用户ID")["是否登录成功"].transform(lambda x: (x == 0).sum())
        df["每分钟失败比例"] = df["登录失败次数"] / df["时间范围分钟"]
        df["登录成功率"] = df.groupby("用户ID")["是否登录成功"].transform("mean")
        df["登录频率"] = df.groupby("用户ID")["登录时间"].transform(lambda x: len(x) / ((x.max() - x.min()).total_seconds() / 60))

        encoder = LabelEncoder()
        df["用户编码"] = encoder.fit_transform(df["用户ID"])
        df["地址编码"] = encoder.fit_transform(df["登录地址"])

        logging.info("数据预处理完成")
        return df

    def apply_rules(self, df):
        """
        应用所有规则到数据
        """
        for rule_name, config in self.rules_config.items():
            try:
                rule_type, param, target_col = config
                add_rule(rule_name, rule_type, param, target_col)
            except ValueError as e:
                logging.error(f"规则 '{rule_name}' 配置无效: {e}")
                raise ValueError(f"规则 '{rule_name}' 配置无效: {e}")
        
        # 应用异常规则
        df = self.anomaly_rules.apply_rules(df)
        return df

    def train_model(self, df):
        """
        模型训练逻辑
        """
        # 示例：使用预处理后的数据进行训练
        model = {"mock_model": "demo"}  # 替换为实际模型训练逻辑
        return model

    def generate_anomaly_statistics(self, df):
        """
        生成异常统计图和报告
        """
        anomaly_count = df["是否异常"].value_counts()
        labels = anomaly_count.index.map(lambda x: "异常登录" if x == 1 else "正常登录").tolist()

        # 绘制饼图
        plt.figure(figsize=(6, 6))
        plt.pie(
            anomaly_count,
            labels=labels,
            autopct="%1.1f%%" if len(anomaly_count) > 1 else None,
            startangle=90,
            colors=["green", "red"][:len(anomaly_count)]
        )
        plt.title("登录行为异常占比")

        # 保存统计图
        plot_file_path = os.path.join(self.imp_folder, "anomalies_plot.png")
        plt.savefig(plot_file_path, format="png")
        return plot_file_path

    def save_anomalies(self, df, encoding):
        """
        保存异常数据到 CSV 文件
        """
        anomalies = df[df["是否异常"] == 1]
        anomalies_file_path = os.path.join(self.imp_folder, "anomalies.csv")
        anomalies.to_csv(anomalies_file_path, index=False, encoding=encoding)
        return anomalies_file_path

    def process_file(self, file):
        """
        主方法：文件加载、预处理、规则应用、模型训练
        """
        df, encoding = self.load_and_preprocess_data(file)

        # 应用规则
        df = self.apply_rules(df)

        # 保存异常数据
        anomalies_file_path = self.save_anomalies(df, encoding)

        # 生成异常统计图
        plot_file_path = self.generate_anomaly_statistics(df)

        # 模型训练（示例）
        model = self.train_model(df)

        return anomalies_file_path, encoding, plot_file_path
